<a href="https://colab.research.google.com/github/Rintaro-Fukui/Calendar-Generator/blob/main/Calendar_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF


apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg


cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

apt-get update
apt-get install chromium chromium-driver

pip install -q selenium ics

Executing: /tmp/apt-key-gpghome.jv8FLbIxEn/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Js1kW4vk9i/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.A7ubJ1sRiF/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [2]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import datetime
import arrow
from ics import Calendar, Event

In [3]:
# ChromeDriverの設定
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome("chromedriver",options=options)
driver.implicitly_wait(10)

# urlを指定
url="https://muscat.musashino-u.ac.jp/portal/top.do"
driver.get(url)

time.sleep(5)

# 学籍番号を入力
login = "s2122098"
driver.find_element(By.XPATH,"//*[@id='userId']").send_keys(login)

# パスワードを入力
password = "Em_3_e5P"
driver.find_element(By.XPATH,"//*[@id='password']").send_keys(password)

# ログインボタンをクリック
btn = driver.find_element(By.XPATH,"//*[@id='loginButton']")
btn.click()

time.sleep(5)

# メニューの非表示を解除
menuList = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[6]/ul/li[2]/ul")
driver.execute_script("arguments[0].setAttribute('style','display: block;')", menuList)

# my時間割にアクセス
myCalendar = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[6]/ul/li[2]/ul/li[1]/a")
myCalendar.send_keys(Keys.ENTER)

# 時間割を抽出
html = driver.page_source
bs = BeautifulSoup(html, "html.parser")
table = str(bs.find_all("table", class_="detail jikanwari_table"))
table = re.findall(r"<!-- コマに対応する時間割情報が存在しない場合、縦幅を確保する -->|<a.*?>(.*?)</a>", table)
jikanwari = [table[0:6], table[6:12], table[12:18], table[18:24], table[24:30], table[30:36], table[36:42]]

In [9]:
jikanwari

[['', '', '', '専門コース演習Ⅰ(人工知能（AI）クリエーション)', '', ''],
 ['', '', 'テキストマイニング', '専門コース演習Ⅰ(人工知能（AI）クリエーション)', '', ''],
 ['データサイエンス特論', '', 'テキストマイニング', '', '未来創造PJ-AⅣ', ''],
 ['データサイエンス特論', '', '社会・環境・ビジネスデザイン', '', '未来創造PJ-AⅣ', ''],
 ['', '', '社会・環境・ビジネスデザイン', '', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '']]

In [10]:
cal = Calendar()
cal.creator = 'Rintaro Fukui'

semester = {
    0: [datetime.date(2023,4,15), datetime.date(2023,6,10)],
    1: [datetime.date(2023,6,15), datetime.date(2023,8,2)],
    2: [datetime.date(2023,9,22), datetime.date(2023,11,13)],
    3: [datetime.date(2023,11,21), datetime.date(2024,1,27)],
}

class_time = {
    0: ['08:50:00', '10:30:00'],
    1: ['10:40:00', '12:20:00'],
    2: ['13:10:00', '14:50:00'],
    3: ['15:00:00', '16:40:00'],
    4: ['16:50:00', '18:30:00'],
    5: ['18:40:00', '20:20:00'],
    6: ['20:20:00', '22:00:00'],
    }

# for i in range(len(semester)):
start = semester[0][0]
stop = semester[0][1]
while start <= stop:
    weekday = start.weekday()
    for i in range(len(jikanwari)):
        rele = jikanwari[i]
        if weekday != 6 and rele[weekday] != '':
            event = Event()
            event.name = jikanwari[i][weekday]
            event.begin = arrow.get(f"{start} {class_time[i][0]}", "YYYY-MM-DD HH:mm:ss").replace(tzinfo="Asia/Tokyo")
            event.end = arrow.get(f"{start} {class_time[i][1]}" ,"YYYY-MM-DD HH:mm:ss").replace(tzinfo="Asia/Tokyo")
            cal.events.add(event)
    start += datetime.timedelta(days=1)

In [13]:
with open('my.ics', 'w', encoding='utf-8') as f:
    f.write(str(cal))

/usr/local/lib/python3.9/dist-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(
